In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from os import environ
from googlemaps import Client
import math
from ipyleaflet import Map, Marker, MarkerCluster, basemap_to_tiles, basemaps, WMSLayer, LayersControl

In [ ]:
connection_url =  environ["ICEURL"]

In [ ]:
status = pd.read_sql(con=connection_url, sql="SELECT * FROM status_daily")
locations = pd.read_sql(con=connection_url, sql="SELECT * FROM locations")
# for col in ['date_first_record', 'date_recent_record', 'created_at', 'updated_at' ]:
#     dataframe2[col] = (new_dataframe[col]).apply(lambda d: pd.to_datetime(str(d)))
# dataframe2.to_sql(name='locations', con=connection_url, if_exists='append', index=False)

In [ ]:
status.sort_values(by=['date'],inplace=True)

In [ ]:
print(pd.io.sql.get_schema(status,'status'))


In [ ]:
locations.head(5)

In [ ]:
sf =  status[status['location_id'].notnull()]

In [ ]:
sf = sf[sf['date'] == sf['date'].max()]

In [ ]:
sf['location_id'] = pd.to_numeric(sf['location_id'], downcast='unsigned')

In [ ]:
status_locations = sf[['id','location_id']].merge(locations[['id','lat','lon']], left_on='location_id', right_on='id')

In [ ]:
del status_locations['id_y']
status_locations.head()

In [ ]:


m = Map(
    center=(39.1, -94.578),
    zoom=4,
    attribution_control=False
)

markers=[]
for index, row in status_locations.iterrows():
    lat, lon = row [['lat','lon']]
    if not math.isnan(lat):
        markers.append(Marker(location=(lat,lon)))

marker_cluster = MarkerCluster(markers=markers)
m.add_layer(marker_cluster)
m.add_layer(basemaps.Esri.DeLorme)
nasa_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-09-17")
m.add_layer(nasa_layer)
m.add_control(LayersControl())
m